In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import netCDF4 as ncdf
import matplotlib as mlab
import matplotlib.pyplot as plt
import xarray as xr
import glob as glob
import os as os
import sys as sys
from file_tools import *
from functools import partial
import time
import multiprocessing
from contextlib import contextmanager
from zipfile import *
import io

# These are 45 vertical levels that the FV3 puts out - use them here to map ARW to that grid for comparison

plevels = np.asarray([100000.,  97500.,  95000.,  92500.,  90000.,  87500.,  85000.,  82500.,
                       80000.,  77500.,  75000.,  72500.,  70000.,  67500.,  65000.,  62500.,
                       60000.,  57500.,  55000.,  52500.,  50000.,  47500.,  45000.,  42500.,
                       40000.,  37500.,  35000.,  32500.,  30000.,  27500.,  25000.,  22500.,
                       20000.,  17500.,  15000.,  12500.,  10000.,   7000.,   5000.,   3000.,
                        2000.,   1000.,    700.,    500.,    200.])

nz_new = plevels.shape[0]

_w_threshold = 5.0
_Rgas       = 287.04
_gravity    = 9.806

# default setting for domain cutting

#sw_corner = None
#ne_corner = None

#ECONUS Domain
sw_corner = (26.0, -105.0)
ne_corner = (49.0, -70.0)

@contextmanager
def poolcontext(*args, **kwargs):
    pool = multiprocessing.Pool(*args, **kwargs)
    yield pool
    pool.terminate()

def hrrr_loop(f,files,sw_corner, ne_corner , writeout, prefix, interpP):
    hrrr_ds, filename = hrrr_grib_read_variable(files[f], sw_corner = sw_corner, ne_corner = ne_corner, writeout=writeout, prefix=prefix, interpP=interpP)

def rrfs_loop(f,files,sw_corner, ne_corner , writeout, prefix):
    rrfs_ds, filename = fv3_grib_read_variable(files[f], sw_corner = sw_corner, ne_corner = ne_corner, writeout=writeout, prefix=prefix)    

In [9]:
#-------------------------------------------------------------------------------------------------------
# Set up file lists for processing
#-------------------------------------------------------------------------------------------------------

#dates = ("2019071918","2020030212","2020050300","2020070700","2020081006")#,"2021052612","2021090100")
dates = ("2019071918","")
hours = np.array([["2200","2300","0000","0100","0200","0300"],""])
for d,date in enumerate(dates):
    print(date)
    # HRRR files

    # hrrr_dir   = '/Users/Louis.Wicker/CAM_Case_Studies/20210526/HRRR'
    # hrrr_files = sorted(glob.glob(hrrr_dir+'/hrrr.*.grib2'))
    hrrr_dir = '/work/larissa.reames/%s/hrrr'%date
    hrrr_files = sorted(glob.glob(hrrr_dir+'/*.grib2'))
    #hrrr_files = ("/work/larissa.reames/2021052612/hrrr/hrrr.t12z.wrfnatf11.grib2","")

    # RRFS1 dir

    #rrfs_dir   = '/Users/Louis.Wicker/CAM_Case_Studies/20210526/RRFS/ctrl'
    #rrfs_files = sorted(glob.glob(rrfs_dir+'/PRSLEV.Grb*'))
    rrfs_dir1 = '/work/larissa.reames/%s/ctrl'%date
    rrfs_files1 = sorted(glob.glob(rrfs_dir1+'/*.grib2'))
    #rrfs_files1 = ("/work/larissa.reames/2021052612/ctrl/PRSLEV.GrbF11.grib2","")

    # RRFS2 dir

    #rrfs_dir   = '/Users/Louis.Wicker/CAM_Case_Studies/20210526/RRFS/ctrl'
    #rrfs_files = sorted(glob.glob(rrfs_dir+'/PRSLEV.Grb*'))
    rrfs_dir2 = '/work/larissa.reames/%s/nord3'%date
    rrfs_files2 = sorted(glob.glob(rrfs_dir2+'/*.grib2'))

    # NAM dir
    nam_dir = '/work/larissa.reames/%s/nam'%date
    nam_files = sorted(glob.glob(nam_dir+'/*.grib2'))
    
    #GRIDRAD_dir
    rad_dir = '/work/larissa.reames/%s/gridrad/raw_files'%date
    rad_files = sorted(glob.glob(rad_dir+'/*.nc'))
    
    #MRMS_dir
    mrms_dir = '/work/larissa.reames/%s/mrms/'%date
    mrms_files = sorted(glob.glob(mrms_dir+'/*top.zip'))

    #regrid_mrms(date,hours[d],mrms_files,'/work/larissa.reames/2021052612/hrrr/econus_202105261200_F09.nc',mrms_dir,
    #            ["EchoTop_30","EchoTop_50","EchoTop_60",'VIL',"MergedReflectivityQC_01.00",
    #             "MergedReflectivityQComposite","HeightCompositeReflectivity","Reflectivity_-10C",
    #            "MergedBaseReflectivityQC"])
    #with poolcontext(processes=6) as pool:
    #    t0 = time.time()
    #    results = pool.map(partial(hrrr_loop, files = hrrr_files,sw_corner = sw_corner, ne_corner = ne_corner, writeout=True, prefix=None, interpP=False ), np.arange(len(hrrr_files)))
    #    t1 = time.time()
    #date,hours,fins,ftarget,weights_file,outdir
    with poolcontext(processes=6) as pool:
        t0 = time.time()
        results = pool.map(partial(regrid_rrfs, hours=hours[d], fins = rrfs_files2,
                                   ftarget='/work/larissa.reames/2021052612/hrrr/econus_202105261200_F09.nc',
                                   weights_file='/work/larissa.reames/rrfsb_to_hrrr_econus_weights.nc',
                                  outdir=rrfs_dir2), np.arange(len(rrfs_files2)))
        t1 = time.time()
    
    #with poolcontext(processes=6) as pool:
    #    t0 = time.time()
    #    results = pool.map(partial(rrfs_loop, files = rrfs_files2, sw_corner = sw_corner, ne_corner = ne_corner, writeout=True, prefix=None), np.arange(len(rrfs_files1)))
    #    t1 = time.time()
    
    

    #for file in nam_files:
    #    nam_ds, filename = fv3_grib_read_variable(file, sw_corner = sw_corner, ne_corner = ne_corner, writeout=True, prefix=None)
    

    #regrid_gridrad(date,rad_files,'/work/larissa.reames/2021052612/hrrr/econus_202105261200_F09.nc')


SyntaxError: invalid syntax (1138419990.py, line 58)

In [7]:
#-------------------------------------------------------------------------------------------------------
# Specify regional domain
# This is tricky, you need to know where to put the domain before you can actually see it easily

# One way is to process a single file perhaps when convection is active, and then choose the domain.  
# Ideally, the domains should be nx > 500, ny > 500, e.g., about 12-15 degrees in both dlat and dlon
#-------------------------------------------------------------------------------------------------------
#

if (date == '20210526'):
# May 26 Case (NW KS) domain
    sw_corner = (30.0, -105.0)
    ne_corner = (43.0, -91.0)

# 19 July 2019 (N Plains) domain
elif (date == '2019071918'):
    sw_corner = (38.0, -97.0)
    ne_corner = (50.0, -82.0)
    
# 02 March 2020 (Upper MS Valley) domain
elif (date == '2020030212'):
    sw_corner = (30.0, -93.0)
    ne_corner = (42.0, -80.0)
    
# 03 May 2020 (Central MS Valley) domain
elif (date == '2020050300'):
    sw_corner = (31.0, -96.0)
    ne_corner = (44.0, -81.0)
    
# 07 July 2020 (Central Plains) domain
elif (date == '2020070700'):
    sw_corner = (38.0, -105.0)
    ne_corner = (50.0, -92.0)
    
# 07 July 2020 (Upper MW) domain
elif (date == '2020081006'):
    sw_corner = (35.0, -100.0)
    ne_corner = (48.0, -86.0)
    
# 26 May 2021 (Central Plains) domain
elif (date == '2021052612'):
    sw_corner = (32, -106.0)
    ne_corner = (44.0, -93.0)

# 01 September 2021 (Mid-Atlantic) domain
elif (date == '2021090100'):
    sw_corner = (32, -82.0)
    ne_corner = (44.0, -69.0)
    
### ALL CASES ECONUS
sw_corner = (26.0, -105.0)
ne_corner = (49.0, -70.0)
    
    

#-------------------------------------------------------------------------------------------------------
# XXX Case


ERROR! Session/line number was not unique in database. History logging moved to new session 546


In [5]:
#-------------------------------------------------------------------------------------------------------
#
# Process the HRRR files - the call to read the grib files returns the xarray data set and the NCDF filename
#
#-------------------------------------------------------------------------------------------------------

for file in hrrr_files:
    hrrr_ds, filename = hrrr_grib_read_variable(file, sw_corner = sw_corner, ne_corner = ne_corner, writeout=True, prefix=None, interpP=True)

------------------------------------------------------------------------------------------------------------------------ 

HRRR Grib READ: Extracting variables from grib file: /work/larissa.reames/2020030212/hrrr/hrrr.t12z.wrfnatf12.grib2 

InterpP is True, Read 3D pressure field from GRIB file

P-max:  103192.10  P-min:  2230.30

Reading my variable:  TEMP from GRIB file variable: Temperature

Reading my variable:  OMEGA from GRIB file variable: Vertical velocity

Reading my variable:  U from GRIB file variable: U component of wind

Reading my variable:  V from GRIB file variable: V component of wind

Reading my variable:  GPH from GRIB file variable: Geopotential Height

Reading my variable:  UH from GRIB file variable: unknown

Reading my variable:  CREF from GRIB file variable: Maximum/Composite radar reflectivity

Reading my variable:  HGT from GRIB file variable: Orography

Creating a sub-region of grid: 30.00, -93.00, 42.00, -80.00 

Successfully wrote new data to file:: /work/l

In [13]:
#-------------------------------------------------------------------------------------------------------
#
# Process the RRFS files - the call to read the grib files returns the xarray data set and the NCDF filename
#
#-------------------------------------------------------------------------------------------------------

for file in rrfs_files1:
    rrfs_ds1, filename = fv3_grib_read_variable(file, sw_corner = sw_corner, ne_corner = ne_corner, writeout=True, prefix=None)

------------------------------------------------------------------------------------------------------------------------ 

RRFS Grib READ: Extracting variables from grib file: /work/larissa.reames/2021052612/ctrl/PRSLEV.GrbF09.grib2 

Reading my variable:  TEMP from GRIB variable: Temperature

Reading my variable:  HGT from GRIB variable: Orography

Reading my variable:  GPH from GRIB variable: Geopotential Height

Reading my variable:  W from GRIB variable: Geometric vertical velocity

Reading my variable:  U from GRIB variable: U component of wind

Reading my variable:  V from GRIB variable: V component of wind

Reading my variable:  UH from GRIB variable: unknown

Reading my variable:  CREF from GRIB variable: Maximum/Composite radar reflectivity

Creating a sub-region of grid: 32.00, -106.00, 44.00, -93.00 

Successfully wrote new data to file:: /work/larissa.reames/2021052612/ctrl/region_202105261200_F09.nc 

------------------------------------------------------------------------

In [14]:
#-------------------------------------------------------------------------------------------------------
#
# Process the RRFS files - the call to read the grib files returns the xarray data set and the NCDF filename
#
#-------------------------------------------------------------------------------------------------------

for file in rrfs_files2:
    rrfs_ds2, filename = fv3_grib_read_variable(file, sw_corner = sw_corner, ne_corner = ne_corner, writeout=True, prefix=None)

------------------------------------------------------------------------------------------------------------------------ 

RRFS Grib READ: Extracting variables from grib file: /work/larissa.reames/2021052612/nord3/PRSLEV.GrbF09.grib2 

Reading my variable:  TEMP from GRIB variable: Temperature

Reading my variable:  HGT from GRIB variable: Orography

Reading my variable:  GPH from GRIB variable: Geopotential Height

Reading my variable:  W from GRIB variable: Geometric vertical velocity

Reading my variable:  U from GRIB variable: U component of wind

Reading my variable:  V from GRIB variable: V component of wind

Reading my variable:  UH from GRIB variable: unknown

Reading my variable:  CREF from GRIB variable: Maximum/Composite radar reflectivity

Creating a sub-region of grid: 32.00, -106.00, 44.00, -93.00 

Successfully wrote new data to file:: /work/larissa.reames/2021052612/nord3/region_202105261200_F09.nc 

----------------------------------------------------------------------

In [23]:
#-------------------------------------------------------------------------------------------------------
#
# Process the NAM files - the call to read the grib files returns the xarray data set and the NCDF filename
#
#-------------------------------------------------------------------------------------------------------

for file in nam_files:
    nam_ds, filename = fv3_grib_read_variable(file, sw_corner = sw_corner, ne_corner = ne_corner, writeout=True, prefix=None)

------------------------------------------------------------------------------------------------------------------------ 

RRFS Grib READ: Extracting variables from grib file: /work/larissa.reames/2021090100/nam/nam.t00z.conusnest.hiresf18.tm00.grib2 

Reading my variable:  TEMP from GRIB variable: Temperature

Reading my variable:  HGT from GRIB variable: Orography

Reading my variable:  GPH from GRIB variable: Geopotential Height

Reading my variable:  W from GRIB variable: Geometric vertical velocity

Reading my variable:  U from GRIB variable: U component of wind

Reading my variable:  V from GRIB variable: V component of wind

Reading my variable:  UH from GRIB variable: unknown

Reading my variable:  CREF from GRIB variable: Maximum/Composite radar reflectivity

Creating a sub-region of grid: 32.00, -82.00, 44.00, -69.00 

Successfully wrote new data to file:: /work/larissa.reames/2021090100/nam/region_2021090100_F18.nc 

----------------------------------------------------------